# Can we predict death?
## Predicting Cancer Survival Time

* Jon Kartago Lamida - 1004796
* Mohamad Ridzuan Yusop - 1004772
* Muhammad Muhaimin Bin Omar - 1004914

## Motivation

WHO published some cancer key facts in 2018. From the facts, we can see the importance of eradicating cancer by improving detection, prediction and treatment. In this project, we want to apply our machine learning knowledge in selecting features and to predict non-small cell lung cancer (NSCLC) patients' survival time using some linear regression models. We will analyze the accuracy of different features and models.

## Notebook Configuration

Update the following configuration to adjust the notebook execution

In [8]:
DATA_VERSION = "v2" # or "v1|v2" v1: Ridzuan Original Data clean up. v2: Jon clean up implementation with removed na.
TEST_SIZE = 0.2
RANDOM_STATE = 42

DATA_URL = "https://wiki.cancerimagingarchive.net/download/attachments/16056854/NSCLC%20Radiomics%20Lung1.clinical-version3-Oct%202019.csv?version=1&modificationDate=1572013183040&api=v2"
DOWNLOAD_PATH = "data.csv"

COLUMN_NAMES = ["PatientID", "Age", "Clinical.T.Stage", "Clinical.N.Stage", "Clinical.M.Stage", "Overall.Stage", 
                "Histology", "Gender", "Survival.Time", "DeadStatus.Event"]

## Import and Download The Data

In [9]:
import pandas as pd
import numpy as np
import os
import tarfile
import urllib.request

def fetch_data(data_url = DATA_URL, download_path = DOWNLOAD_PATH):
    urllib.request.urlretrieve(data_url, download_path)

def load_data(download_path = DOWNLOAD_PATH):
    return pd.read_csv(download_path)

fetch_data()
raw_data = load_data()

raw_data.columns = COLUMN_NAMES

## Understand The Data

In [10]:
print(raw_data.head())
print(raw_data.info())

   PatientID      Age  Clinical.T.Stage  Clinical.N.Stage  Clinical.M.Stage  \
0  LUNG1-001  78.7515               2.0                 3                 0   
1  LUNG1-002  83.8001               2.0                 0                 0   
2  LUNG1-003  68.1807               2.0                 3                 0   
3  LUNG1-004  70.8802               2.0                 1                 0   
4  LUNG1-005  80.4819               4.0                 2                 0   

  Overall.Stage                Histology Gender  Survival.Time  \
0          IIIb               large cell   male           2165   
1             I  squamous cell carcinoma   male            155   
2          IIIb               large cell   male            256   
3            II  squamous cell carcinoma   male            141   
4          IIIb  squamous cell carcinoma   male            353   

   DeadStatus.Event  
0                 1  
1                 1  
2                 1  
3                 1  
4                 

In [11]:
# Check Is there any  Non Avialable Data 
display(raw_data.isna().any())

PatientID           False
Age                  True
Clinical.T.Stage     True
Clinical.N.Stage    False
Clinical.M.Stage    False
Overall.Stage        True
Histology            True
Gender              False
Survival.Time       False
DeadStatus.Event    False
dtype: bool

In [12]:
# Removing Non Availabile Datas from the raw
raw_data = raw_data.fillna(-1)

In [13]:
# Check Is there any Non Available Data 
display(raw_data.isna().any())

PatientID           False
Age                 False
Clinical.T.Stage    False
Clinical.N.Stage    False
Clinical.M.Stage    False
Overall.Stage       False
Histology           False
Gender              False
Survival.Time       False
DeadStatus.Event    False
dtype: bool

In [14]:
# Now lets check the class distributions
import seaborn as sns
raw_data["DeadStatus.Event"].value_counts()
sns.countplot(x="DeadStatus.Event",data=raw_data)

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
raw_data['Survival.Time'].value_counts()

In [ ]:
raw_data['Histology'].value_counts()
sns.countplot(x="Histology",data=raw_data)

In [ ]:
raw_data['Gender'].value_counts()
sns.countplot(x="Gender",data=raw_data)

In [ ]:
pd.crosstab(index=raw_data['Gender'], columns=raw_data['DeadStatus.Event'])

In [ ]:
pd.crosstab(index=raw_data['Gender'], columns=raw_data['Clinical.T.Stage'])

In [ ]:
pd.crosstab(index=raw_data['Gender'], columns=raw_data['DeadStatus.Event'])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt  
sns.boxplot(x=raw_data.Age)

In [ ]:
raw_data["Clinical.T.Stage"].value_counts()
sns.countplot(x="Clinical.T.Stage",data=raw_data)

In [ ]:
raw_data["Clinical.M.Stage"].value_counts()
sns.countplot(x="Clinical.M.Stage",data=raw_data)
# There are too many 0 stage in this attribute. We might need to drop this feature.

In [ ]:
raw_data["Clinical.N.Stage"].value_counts()
sns.countplot(x="Clinical.N.Stage",data=raw_data)

In [ ]:
#Find the correlation of each coffecients
raw_data.corr()

In [ ]:
# Correlation using pandas
from pandas.plotting import scatter_matrix

scatter_matrix(raw_data, figsize=(40, 10))

## Scatterplot of The Dataset

In [ ]:
# Plot
raw_data.plot.scatter(x='Age', y='Survival.Time', alpha=0.5)
# Need to remove age 0

# Data Cleanup
## Data V1

In [ ]:
# make dictionary for string values
string_val_keys = ['Overall.Stage', 'Histology', 'Gender']
dictionary = {}
for key in string_val_keys:
    classes = list(raw_data[key].unique())
    if -1 in classes:
        classes.remove(-1)
    dictionary[key] = classes

# Seperate independent variable named as data and dependent variable name lablels
data = []
survival_times = []
# deadstatus.event
dead_status_events = []
for i in range(len(raw_data.values)):
    if raw_data['DeadStatus.Event'][i] == -1:
        continue
    data_line = []
    for key in raw_data.keys()[1:]:
        if raw_data[key][i] == -1:
            data_line.append(-1)
            continue
        if key in string_val_keys:
            data_line.append(dictionary[key].index(raw_data[key][i]))
        else:
            data_line.append(raw_data[key][i])
    data.append(data_line[:-2])
    # Target is the last two columns
    survival_times.append(data_line[-2:-1][0]) # Survival.time
    dead_status_events.append(data_line[-1]) # deadstatus.event
print('There are %d items in the given dataset' % len(data))

## Data V2

In [ ]:
stage_mapping = {
    "I": 1,
    "II": 2,
    "IIIa": 3,
    "IIIb": 4
}
gender_mapping = {
    "male": 0,
    "female": 1
}
histology_mapping = {
    'large cell': 0,
    'squamous cell carcinoma': 1,
    'adenocarcinoma': 2,
    'nos': 3
}

data_v2 = raw_data.copy()
data_v2["Overall.Stage"] = raw_data[["Overall.Stage"]].applymap(lambda x: stage_mapping.get(x))
data_v2["Gender"] = raw_data[["Gender"]].applymap(lambda x: gender_mapping.get(x))
data_v2["Histology"] = raw_data[["Histology"]].applymap(lambda x: histology_mapping.get(x))

data_v2 = data_v2.dropna()
survival_times_v2 = data_v2[['Survival.Time']].values.ravel()
dead_status_events_v2 = data_v2[['DeadStatus.Event']].values.ravel()
# Remove PatientID and labels
data_v2 = data_v2.iloc[:, 1:-2].values
data_v2
print('There are %d items in the given dataset' % len(data_v2))

## Train Test Split

In [ ]:
#Split the data nd labels set into 80% as training and 20% as testing
from sklearn.model_selection import train_test_split

def run_train_test_split(data_version = DATA_VERSION, test_size = TEST_SIZE, random_state = RANDOM_STATE):
    if (DATA_VERSION == "v1"):
        X_train, X_test, y_train_survival_times, y_test_survival_times = train_test_split(data, survival_times, test_size=test_size, random_state=random_state)
        X_train, X_test, y_train_dead_status_events, y_test_dead_status_events = train_test_split(data, dead_status_events, test_size=test_size, random_state=random_state)
    else:
        X_train, X_test, y_train_survival_times, y_test_survival_times = train_test_split(data_v2, survival_times_v2, test_size=test_size, random_state=random_state)
        X_train, X_test, y_train_dead_status_events, y_test_dead_status_events = train_test_split(data_v2, dead_status_events_v2, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train_survival_times, y_train_dead_status_events, y_test_survival_times, y_test_dead_status_events

X_train, X_test, y_train_survival_times, y_train_dead_status_events, y_test_survival_times, y_test_dead_status_events = run_train_test_split()
print(f'There are {len(X_train)} items in the train dataset')
print(f'There are {len(X_test)} items in the test dataset')

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_sd = sc.fit_transform(X_train)
X_test_sd = sc.transform(X_test)

# Predicting DeadStatus.Event Classification

In [ ]:
classifications = []

## Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report


clf = LogisticRegression(random_state = 0)

clf.fit(X_train_sd, y_train_dead_status_events)
y_pred = clf.predict(X_test_sd)

cm = confusion_matrix(y_test_dead_status_events, y_pred)
report = classification_report(y_test_dead_status_events, y_pred)
acc = accuracy_score(y_test_dead_status_events, y_pred)*100

classifications.append({
    "name": clf.__class__.__name__,
    "cm": cm,
    "acc": acc,
    "report": report
})

In [ ]:
import matplotlib.pyplot as plt  
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set()
plt.figure(figsize = (10,7))
sns.heatmap(cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

## K-NN Algorithom

Steps
1.Create new data set without categorical variable and assigned name as data
2.Create new data set Only with categorical variable and assigned name as lables
3.Split the data set into 80% as Training and assigned name as x_train as well as 20% as Testing & Assigned name as x_test
4.Split the labels set into 80% as Training and assigned name as y_train as well as 20 as Testing & Assigned name as y_test


In [ ]:
#Normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler ()
x_train_normal = scaler.fit_transform(X_train)
x_test_normal = scaler.fit_transform(X_test)

In [ ]:
from sklearn import neighbors

n_neighbors = 10
weights = 'uniform'

clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
clf.fit(x_train_normal, y_train_dead_status_events)
y_pred = clf.predict(x_test_normal)
cm = confusion_matrix(y_test_dead_status_events, y_pred)
report = classification_report(y_test_dead_status_events, y_pred)
acc = accuracy_score(y_test_dead_status_events, y_pred)*100

classifications.append({
    "name": clf.__class__.__name__,
    "cm": cm,
    "acc": acc,
    "report": report
})

In [ ]:
import matplotlib.pyplot as plt  
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set()
plt.figure(figsize = (10,8))
sns.heatmap(cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf = clf.fit(X_train_sd,y_train_dead_status_events)
clf.fit(X_train_sd, y_train_dead_status_events)
y_pred = clf.predict(X_test_sd)
cm = confusion_matrix(y_test_dead_status_events, y_pred)
report = classification_report(y_test_dead_status_events, y_pred)
acc = accuracy_score(y_test_dead_status_events, y_pred)*100

classifications.append({
    "name": clf.__class__.__name__,
    "cm": cm,
    "acc": acc,
    "report": report
})


In [ ]:
import matplotlib.pyplot as plt  
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set()
plt.figure(figsize = (10,7))
sns.heatmap(cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

## Naive Bayes Model


In [ ]:
# Training the Naive Bayes model on the Training set
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train_sd, y_train_dead_status_events)
y_pred = clf.predict(X_test_sd)
cm = confusion_matrix(y_test_dead_status_events, y_pred)
report = classification_report(y_test_dead_status_events, y_pred)
acc = accuracy_score(y_test_dead_status_events, y_pred)*100

classifications.append({
    "name": clf.__class__.__name__,
    "cm": cm,
    "acc": acc,
    "report": report
})




In [ ]:
plt.figure(figsize = (10,7))
sns.heatmap(cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

## RandomForest Model

In [ ]:
# Training the Random Forest Classification model on the Training set
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
clf.fit(X_train_sd, y_train_dead_status_events)
y_pred = clf.predict(X_test_sd)
cm = confusion_matrix(y_test_dead_status_events, y_pred)
report = classification_report(y_test_dead_status_events, y_pred)
acc = accuracy_score(y_test_dead_status_events, y_pred)*100

classifications.append({
    "name": clf.__class__.__name__,
    "cm": cm,
    "acc": acc,
    "report": report
})


In [ ]:
plt.figure(figsize = (10,7))
sns.heatmap(cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

## Support Vector Machine (SVM)

In [ ]:
# Training the SVM model on the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
clf.fit(X_train_sd, y_train_dead_status_events)
y_pred = clf.predict(X_test_sd)
cm = confusion_matrix(y_test_dead_status_events, y_pred)
report = classification_report(y_test_dead_status_events, y_pred)
acc = accuracy_score(y_test_dead_status_events, y_pred)*100

classifications.append({
    "name": clf.__class__.__name__,
    "cm": cm,
    "acc": acc,
    "report": report
})


In [ ]:
plt.figure(figsize = (10,7))
sns.heatmap(cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

# Compare All Models

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [ ]:
#Comparing classifiers via voting
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()
voting_clf = VotingClassifier(
estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
voting='hard'
)
voting_clf.fit(X_train_sd, y_train_dead_status_events)

In [ ]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train_sd, y_train_dead_status_events)
    y_pred = clf.predict(X_test_sd)
    print(clf.__class__.__name__, accuracy_score(y_test_dead_status_events, y_pred))

# Predicting DeadStatus.Event Classification Summary


In [ ]:
from textwrap import dedent

for record in classifications:
    print(dedent(f"""
    Model: {record["name"]}
    Confusion Matrix: 
    {str(record["cm"]).strip()}
    Accuracy: {record["acc"]}
    Report: 
    {record["report"]}
    """))
    


I have used Logistic Regression, KNN, Decission Tree, Random Forest and SVM for analysis the data 

## Accuracy of Each model
* Accuracy of Logistic Regression =89.41176470588236
* Accuracy of knn=87.05882352941177
* Accuracy of Decision Tree=89.41176470588236
* Accuracy of Naive Bayes =85.88235294117646
* Accuracy of Random Forest=85.88235294117646
* Accuracy Of SVM=85.88235294117646

Here, Logistic Regression model Accuracy higher than other Machine learning model, so we could suggest Logistic Regression Model best for this Prediction 

## Advantage of Logistic Regression Model for This Analysis
* It has good Accuracy among them Moreover#
* It has Good sensitivity to predict which pation has deadstatus.event ==1 So, It's good model for that

# Predicting Survival.Time

In [ ]:
from sklearn.metrics import max_error, mean_squared_error, mean_absolute_error, median_absolute_error, explained_variance_score, r2_score

def regression_metrics(X_train, y_test_survival_times, y_pred):
    #print(f"Coefficients: {model.coef_}")
    return dedent(f"""
    Max error: {max_error(y_test_survival_times, y_pred)} 
    Mean Squared Error: {mean_squared_error(y_test_survival_times, y_pred)}
    Mean Absolute Error: {mean_absolute_error(y_test_survival_times, y_pred)}
    Median Absolute Error: {median_absolute_error(y_test_survival_times, y_pred)}
    Explained Variance Score: {explained_variance_score(y_test_survival_times, y_pred)}""")
    #for i in range(len(y_test_survival_times)):
    #    print(f"features: {X_test[i]} actual: {y_test_survival_times[i]} predicted: {y_pred[i]}")

In [ ]:
regressions = []

## LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train_survival_times)
y_pred = model.predict(X_test)


regressions.append({
    "name": model.__class__.__name__,
    "metrics": regression_metrics(X_train, y_test_survival_times, y_pred)
})

## RidgeRegression

In [ ]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=.5)
model.fit(X_train, y_train_survival_times)
y_pred = model.predict(X_test)

regressions.append({
    "name": model.__class__.__name__,
    "metrics": regression_metrics(X_train, y_test_survival_times, y_pred)
})

## Lasso

In [ ]:
from sklearn.linear_model import Lasso
model = Lasso(alpha=.5)
model.fit(X_train, y_train_survival_times)
y_pred = model.predict(X_test)

regressions.append({
    "name": model.__class__.__name__,
    "metrics": regression_metrics(X_train, y_test_survival_times, y_pred)
})

## BayesianRidge

In [ ]:
from sklearn.linear_model import BayesianRidge
model = BayesianRidge()
model.fit(X_train, y_train_survival_times)
y_pred = model.predict(X_test)

regressions.append({
    "name": model.__class__.__name__,
    "metrics": regression_metrics(X_train, y_test_survival_times, y_pred)
})

## PolynomialRegression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

model =  Pipeline([('poly', PolynomialFeatures(degree=3)),
                   ('linear', LinearRegression(fit_intercept=False))])
model.fit(X_train, y_train_survival_times)
y_pred = model.predict(X_test)

regressions.append({
    "name": model.__class__.__name__,
    "metrics": regression_metrics(X_train, y_test_survival_times, y_pred)
})

## SVM Regression

In [ ]:
from sklearn.svm import SVR

for kernel in ["linear", "rbf"]:    
    model = SVR(kernel = kernel)
    model.fit(X_train, y_train_survival_times)
    y_pred = model.predict(X_test)
    
    regressions.append({
        "name": model.__class__.__name__ + "_" + kernel,
        "metrics": regression_metrics(X_train, y_test_survival_times, y_pred)
    })


## KNN Regressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

for weights in ['uniform', 'distance']:
    model = KNeighborsRegressor(n_neighbors=5, weights=weights)
    model.fit(X_train, y_train_survival_times)
    y_pred = model.predict(X_test)
    regressions.append({
        "name": model.__class__.__name__ + "_" + weights ,
        "metrics": regression_metrics(X_train, y_test_survival_times, y_pred)
    })

## DecisionTree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=10)
model.fit(X_train, y_train_survival_times)
y_pred = model.predict(X_test)

regressions.append({
    "name": model.__class__.__name__,
    "metrics": regression_metrics(X_train, y_test_survival_times, y_pred)
})

## AdaBoostRegressor

In [ ]:
# Training the Logistic Regression model on the Training set
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

rng = np.random.RandomState(1)
model = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4), n_estimators=300, random_state=rng)
model.fit(X_train, y_train_survival_times)
y_pred = model.predict(X_test)

regressions.append({
    "name": model.__class__.__name__,
    "metrics": regression_metrics(X_train, y_test_survival_times, y_pred)
})

# Predicting Survival.Time Regression Summary


In [ ]:
for record in regressions:
    print("Model: " + record["name"], record["metrics"])
    print()